In [30]:
%matplotlib inline
import os
import sys
import itertools
import re
import pprint
import imp

import xray
import numpy as np
import pandas as pd

sys.path.append('/nuwa_cluster/home/jackyu/climatools/')
import climatools.misc as climamisc
imp.reload(climamisc)

<module 'climatools.misc' from '/nuwa_cluster/home/jackyu/climatools/climatools/misc.py'>

## Subroutine relations

In [27]:
def fortran_files():
    dir_cam = '/nuwa_cluster/home/jackyu/climate_models/cesm1_2_2/models/atm/cam/src/physics/cam/'
    cam_files = ['aer_rad_props.F90', 
                 'modal_aer_opt.F90',
                 'rad_constituents.F90', 
                 'phys_prop.F90', 
                 'physpkg.F90']
    fpaths = [os.path.join(dir_cam, file) for file in cam_files]
    return fpaths

In [28]:
pprint.pprint(climamisc.Fortran_subroutine_relations_from_files(paths_fortran = fortran_files()))

{'aer_optics_log': {'childs': [], 'parents': []},
 'aer_optics_log_rh': {'childs': [], 'parents': ['bulk_props_init']},
 'aer_rad_props_init': {'childs': ['rad_cnst_get_info',
                                   'addfld',
                                   'add_default',
                                   'phys_getopts'],
                        'parents': ['phys_init']},
 'aer_rad_props_lw': {'childs': ['modal_aero_lw',
                                 'qsat',
                                 'rad_cnst_get_info',
                                 'pbuf_get_field',
                                 'rad_cnst_get_aer_mmr',
                                 'rad_cnst_get_aer_props',
                                 'endrun'],
                      'parents': []},
 'aer_rad_props_sw': {'childs': ['get_volcanic_rad_props',
                                 'get_hygro_rad_props',
                                 'qsat',
                                 'get_nonhygro_rad_props',
                 

## Aerosol data file read

In [32]:
def aer_dir():
    return '/nuwa_data/data/cesm1/inputdata/atm/cam/physprops/'

def nc_files_read_in_physprop_init():
    filelist = [('mam3_mode1_rrtmg_c110318.nc', 'modal'),
                ('mam3_mode2_rrtmg_c110318.nc', 'modal'),
                ('mam3_mode3_rrtmg_c110318.nc', 'modal'),
                ('sulfate_rrtmg_c080918.nc', 'hygroscopic'),
                ('ocpho_rrtmg_c101112.nc', 'insoluble'),
                ('ocphi_rrtmg_c100508.nc', 'hygroscopic'),
                ('bcpho_rrtmg_c100508.nc', 'insoluble'),
                ('dust4_rrtmg_c090521.nc', 'insoluble'),
                ('ssam_rrtmg_c100508.nc', 'hygroscopic')]

In [40]:
# mam3_mode1

print('''
Note that because Fortran fills its arrays' elements with a different order, the order of the dimensions
displayed here by xray are opposite of those in the Fortran code --- transpose.
''')

with xray.open_dataset(
    os.path.join(aer_dir(), 'mam3_mode1_rrtmg_c110318.nc')) as ds:
    print(ds)


Note that because Fortran fills its arrays' elements with a different order, the order of the dimensions
displayed here by xray are opposite of those in the Fortran code --- transpose.

<xray.Dataset>
Dimensions:           (coef_number: 5, lw_band: 16, mode: 1, refindex_im: 10, refindex_real: 7, sw_band: 14)
Coordinates:
  * coef_number       (coef_number) int64 0 1 2 3 4
  * lw_band           (lw_band) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
  * mode              (mode) int64 0
  * refindex_im       (refindex_im) int64 0 1 2 3 4 5 6 7 8 9
  * refindex_real     (refindex_real) int64 0 1 2 3 4 5 6
  * sw_band           (sw_band) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13
Data variables:
    absplw            (lw_band, mode, refindex_im, refindex_real, coef_number) float64 0.0 ...
    abspsw            (sw_band, mode, refindex_im, refindex_real, coef_number) float64 0.0 ...
    asmpsw            (sw_band, mode, refindex_im, refindex_real, coef_number) float64 0.8877 ...
    extpsw       

# modal_aero_sw()

subroutine modal_aero_sw() in modal_aer_opt.F90

Dimensions:

+ pcols --- number of columns/soundings
+ pver --- number of model/atm layers
+ nswbands --- number of shortwave spectral bands (= 14)
+ ncoef --- number of coeffiecients (= 5)
+ prefr --- number of real refractive indices (= 7)
+ prefi --- number of imaginary refractive indices (= 10)

Notes:

+ Static variables are coloured green.
+ Dimension indicators are read as such.  For example, *specmmr(pcols, pver: m, l)*. *specmmr* is an array of dimensions *pcols* x *pver* and it corresponds to aerosol mode *m* and species *l*.

---------------------------------------------------------------------
Pseudo code

+ Get layer mass, *mass(pcols, pver)*, from model state.
+ Get air density, *air_density(pcols, pver)*, from model state.
+ Get number of modes, *nmodes*, for *list_idx*.
+ Get number mode wet diametre for all modes, *dgnumwet_m(pcols, pver, nmodes)* and aerosol water for all modes, *qaerwat_m(pcols, pver, nmodes)[g/g]*, from physical buffer. 



+ **Start loop over modes. do m = 1, *nmodes***
+ Get the following properties from initialised table, *physprop*:

   * geometric standard deviation of number distribution, <span style="color: green">*sigma_logr_aer(: m)*</span>
   * table of real refractive indices for aerosols, <span style="color: green">*refrtabsw(prefr, nswbands: m)*</span>
   * table of imag refractive indices for aerosols, <span style="color: green">*refitabsw(prefi, nswbands: m)*</span>
   * specific extinction, <span style="color: green">*extpsw(ncoef, prefr, prefi, nswbands: m)*</span>
   * specific absorption, <span style="color: green">*abspsw(ncoef, prefr, prefi, nswbands: m)*</span>
   * asymmetry factor, <span style="color: green">*asmpsw(ncoef, prefr, prefi, nswbands: m)*</span>
   
+ Get the number species, *nspec(: m)*, from initialised table, *modes*.
+ Calculate the following size parametres using *pcols*, *sigma_logr_aer(: m)* and *dgnumwet(pcols, pver: m)*

   * aerosol surface mode radius, *radsurf(pcols, pver: m)*
   * log aerosol surface mode radius, *logradsurf(pcols, pver: m)*
   * *cheb(ncoef, pcols, pver: m)*
   


+ **Start loop over spectral bands. do isw = 1, nswbands**
+ **Start loop over layers. do k = top_lev, pver**
+ **Start loop over aerosol species of this mode. do l = 1, nspec**


+ Get species mass mixing ratio, *specmmr(pcols, pver: m, l)*, from either *state* or *pbuf*.
+ Get the following from initialised table *physprop*:

   * species density, <span style="color: green">*specdens(: m, l)[kg/m3]*</span>
   * species refractive index, <span style="color: green">*specrefindex(nswbands: m, l)*</span>
   * species type, <span style="color: green">*spectype(: m, l)*</span>
   * <span style="color: green">*hygro_aer(: m, l)*</span>
   
+ Get volume concentration of aerosol, *vol(pcols: m, l, k)* from *specmmr(pcols, pver: m, l)* and *specdens(: m, l)[kg/m3]*.
+ Add above to volume concentration of aerosol for all species, *dryvol(pcols: m, k)*.
+ Add above to complex refractive index for all species, *crefin(pcols: m, k)*.
+ **End loop over aerosol species of this mode.**


+ Get volume concentration of water, *watervol(pcols: k, m)*, from *qaerwat(ncol, pver: m)* and *rhoh20*.
+ Get volume concentration of wet, *wetvol(pcols: k, m)[m3/kg]*.
+ Use *wetvol(pcols: k, m)[m3/kg]* and complex refractive index for water visible,  *crefwsw(nswbands)*, to change *crefin(pcols: m, k)*, which becomes *crefin(pcols: m, k, isw)*.
+ Split *crefin(pcols: m, k, isw)* into real and imaginary parts, *refr(pcols: m, k, isw)* and *refi(pcols: m, k, isw)*.


+ Get, using bilinear interpolation, *cext(pcols, ncoef: m, k, isw)*, from *extpsw(ncoef, prefr, prefi, nswbands)*, *refr(pcols: m, k, isw)*, *refi(pcols: m, k, isw)*, *refrtabsw(prefr, nswbands: m)* and *refitabsw(prefi, nswbands: m)*.
+ Get, using bilinear interpolation, *cabs(pcols, ncoef: m, k, isw)*, from *abspsw(ncoef, prefr, prefi, nswbands)*, *refr(pcols: m, k, isw)*, *refi(pcols: m, k, isw)*, *refrtabsw(prefr, nswbands: m)* and *refitabsw(prefi, nswbands: m)*.
+ Get, using bilinear interpolation, *casm(pcols, ncoef: m, k, isw)*, from *asmpsw(ncoef, prefr, prefi, nswbands)*, *refr(pcols: m, k, isw)*, *refi(pcols: m, k, isw)*, *refrtabsw(prefr, nswbands: m)* and *refitabsw(prefi, nswbands: m)*.


+ Get parameterised specific extinction, *pext(pcols: m, k, isw)* from *cext(pcols, ncoef: m, k, isw)* and *cheb(ncoef, pcols, pver: m)*
+ Get parameterised specific extinction, *pext(pcols: m, k, isw)[m2/kg aerosol]*, from *pext(pcols: m, k, isw)[m2/kg water]*, *wetvol(pcols: k, m)[m3/kg]* and *rhoh20*.
+ Get parameterised specific absorption, *pabs(pcols: m, k, isw)[m2/kg]*, from *cabs(pcols, ncoef: m, k, isw)* and *cheb(ncoef, pcols, pver: m)*.
+ Get parameterised asymmetry factor, *pasm(pcols: m, k, isw)*, from *casm(pcols, ncoef: m, k, isw)* and *cheb(ncoef, pcols, pver: m)*.
+ Adjust *pabs(pcols: m, k, isw)* using *wetvol(pcols: k, m)[m3/kg]* and *rhoh20*.



+ Get parameterised single scattering albedo, *palb(pcols: m, k, isw)*, from *pabs(pcols: m, k, isw)* and *pext(pcols: m, k, isw)*. <span style="color: red">repeated in the code?</span>
+ Get aerosol optical depth, *dopaer(pcols: m, k, isw)*, from *pext(pcols: m, k, isw)[m2/kg aerosol]* and *mass(pcols, pver)*.



+ Add to aerosol optical depth, *tauxar(pcols, k, isw)*, *dopaer(pcols: m, k, isw)*.
+ Add to aerosol single scattering albedo, *wa(pcols, k, isw)*,  *dopaer(pcols: m, k, isw) x palb(pcols: m, k, isw)*.
+ Add to aerosol asymmetry factor, *ga(ncol, k, isw)*, *dopaer(pcols: m, k, isw) x palb(pcols: m, k, isw) x pasm(pcols: m, k, isw)*.
+ Add to forward scattering factor, *fa(ncol, k , isw)*, *dopaer(pcols: m, k, isw) x palb(pcols: m, k, isw) x pasm(pcols: m, k, isw) x pasm(pcols: m, k, isw)*.


+ **end loop over layers**
+ **end loop over spectral bands**
+ **end loop over nmodes**